In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
import classifier as cf
import seaborn as sns
sns.set_style("darkgrid")

In [2]:
traindata = pd.read_csv("data/processed_training_data.csv")
traindata.head()

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,click_bool,booking_bool,score
0,0,1,2013-04-04 08:32:15,12,187,219,893,3,3.5,1,...,1,0,4,0,1,1,1,0,0,0
1,1,1,2013-04-04 08:32:15,12,187,219,10404,4,4.0,1,...,1,0,4,0,1,1,1,0,0,0
2,2,1,2013-04-04 08:32:15,12,187,219,21315,3,4.5,1,...,1,0,4,0,1,1,1,0,0,0
3,3,1,2013-04-04 08:32:15,12,187,219,27348,2,4.0,1,...,1,0,4,0,1,1,1,0,0,0
4,4,1,2013-04-04 08:32:15,12,187,219,29604,4,3.5,1,...,1,0,4,0,1,1,1,0,0,0


In [3]:
testdata = pd.read_csv("data/processed_testing_data.csv")
testdata.head()

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,score
0,0,1,2013-02-02 15:27:40,24,216,219,3180,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
1,1,1,2013-02-02 15:27:40,24,216,219,5543,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
2,2,1,2013-02-02 15:27:40,24,216,219,14142,2,3.5,1,...,0,19222,1,10,2,0,1,0,0,0
3,3,1,2013-02-02 15:27:40,24,216,219,22393,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0
4,4,1,2013-02-02 15:27:40,24,216,219,24194,3,4.5,1,...,0,19222,1,10,2,0,1,0,0,0


In [4]:
print(traindata.columns)

Index(['Unnamed: 0', 'srch_id', 'date_time', 'site_id',
       'visitor_location_country_id', 'prop_country_id', 'prop_id',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'random_bool', 'click_bool', 'booking_bool', 'score'],
      dtype='object')


In [5]:
attrs = ["prop_starrating", "prop_location_score1", "price_usd"]
y = traindata["score"]
X = traindata[attrs]
flag = "True"
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
gnb = cf.gaussian_naive_bayes(Xtrain, Xtest, ytrain, ytest, flag)

Gaussian Naive Bayes: cross vali score 0.9722290592254431  and accuracy score 0.9719683880862768


In [7]:
dt = cf.decision_tree(Xtrain, Xtest, ytrain, ytest, flag)

Decision Tree: cross vali score 0.9628134789466961  and accuracy score 0.9624865426558195


In [ ]:
knn = cf.k_nearest_neighbours(Xtrain, Xtest, ytrain, ytest, flag)

In [ ]:
svm = cf.support_vector_machine(Xtrain, Xtest, ytrain, ytest, flag)

In [6]:
sgd = cf.stochastic_gradient_descent(Xtrain, Xtest, ytrain, ytest, flag)

Stochastic Gradient Descent: cross vali score 0.9177282060653934  and accuracy score 0.9590531791985233


In [ ]:
ab = cf.ada_boost(Xtrain, Xtest, ytrain, ytest, flag)